In [ ]:
!pip install -q leafmap
!pip install -q geopandas

In [ ]:
import pandas as pd
import leafmap.foliumap as leafmap
import geopandas
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import re

def make_numeric(s):
  return re.sub('[^0-9.-]', '', s)

def make_useful_string(s):
  return re.sub('[^:print:]', '', s)

In [ ]:
tox = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/arch_ai_final_project/datasets/toxic_sites.xlsx - Sheet1.csv')

In [ ]:
tox['Name'] = tox['Site Name'].astype(np.str)
tox['City'] = tox['City'].astype(np.str)
tox['Zip'] = tox['Zip'].astype(np.str)
tox['County'] = tox['County'].astype(np.str)
tox['Site ID'] = (tox.apply(lambda x: make_numeric(x['Site ID']), axis=1)).astype(np.int32)
tox['Latitude'] = (tox.apply(lambda x: make_numeric(x['Latitude']), axis=1)).astype(np.float64)
tox['Longitude'] = (tox.apply(lambda x: make_numeric(x['Longitude']), axis=1)).astype(np.float64)


new_tox = tox.drop(columns=['Region', 'State', 'Unnamed: 4', 'EPA ID', 'Address', 'FF Ind', 'NAI', 'Native American Entity ', 'lat', 'lat1', 'long1', 'long'], axis=1)

In [ ]:
import json

pollution = geopandas.GeoDataFrame.from_features(json.load(open('/content/drive/MyDrive/Colab Notebooks/arch_ai_final_project/datasets/worldwide-pollution.geojson')))

In [ ]:
pollution['latitude'] = pollution.apply(lambda x: x['geopoint'][0], axis=1).astype(np.float64)
pollution['longitude'] = pollution.apply(lambda x: x['geopoint'][1], axis=1).astype(np.float64)
pollution['value_no2'] = pollution['value_no2'].astype(np.float64)
pollution['value_o3'] = pollution['value_o3'].astype(np.float64)
pollution['value_co'] = pollution['value_co'].astype(np.float64)

no2 = pollution[['value_no2', 'latitude', 'longitude']]
o3 = pollution[['value_o3', 'latitude', 'longitude']]
co = pollution[['value_co', 'latitude', 'longitude']]

In [ ]:
def crs(gdf):
  gdf.set_crs(epsg=4326, inplace=True)
  return gdf.to_crs(epsg=4326)

def colorize(feature):
  grade = feature['holc_grade']
  color = 'ff0000'
  if grade == 'A':
    color = '00ff00'
  elif grade == 'B':
    color = '0000ff'
  elif grade == 'C':
    color = 'ffff00'
  return {'fillColor' : color}

In [ ]:
redlining_geo = '/content/drive/MyDrive/Colab Notebooks/arch_ai_final_project/datasets/redlining_map.geojson'
toxic_sites_geo = crs(geopandas.GeoDataFrame(new_tox, geometry=geopandas.points_from_xy(new_tox['Longitude'], new_tox['Latitude'])))
lead_geo = '/content/drive/MyDrive/Colab Notebooks/arch_ai_final_project/datasets/lead.csv'
world = leafmap.Map()


In [ ]:
world.add_geojson(redlining_geo, layer_name='Redlined Districts')
world.add_gdf(toxic_sites_geo, layer_name='Toxic Sites')
world.add_heatmap(no2, latitude='latitude', longitude='longitude', value='value_no2',  name='NO2 Pollution')
world.add_heatmap(o3, latitude='latitude', longitude='longitude', value='value_o3',  name='O3 Pollution')
world.add_heatmap(co, latitude='latitude', longitude='longitude', value='value_co',  name='CO Pollution')

world.add_heatmap(lead_geo, latitude='Latitude', longitude='Longitude', value='Pb Content (mg/kg)', name='Lead Pollution Levels')

In [ ]:
world.to_html("world.html")